<a href="https://colab.research.google.com/github/evinracher/3010090-ontological-engineering/blob/main/week2/2_05_Tools_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Tools en LangChain 1.0 con Gemini

Formato: Google Colab

## 1. Configuración de la API Key

In [ ]:
from google.colab import userdata
import os

# Obtener la API key desde userdata
api_key = userdata.get('GOOGLE_API_KEY')

# Opcional: Guardarla como variable de entorno
os.environ['GOOGLE_API_KEY'] = api_key

# Verificar que se haya cargado correctamente
print("API Key cargada:", "Sí" if api_key else "No")
print("Primeros caracteres:", api_key[:10] if api_key else "No encontrada")



## 2. Instalación del entorno

In [ ]:
!pip install -q -U langchain-google-genai

In [ ]:
pip show langchain

## 3. Importar de librerías

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
import math

## 4. Tool simple: raiz cuadrada

In [ ]:
@tool
def calcular_raiz(num: float) -> float:
    """Calcula la raíz cuadrada de un número."""
    return math.sqrt(num)

### 5. Crear e invocar modelo con Tool

In [ ]:
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage, SystemMessage

In [ ]:
# Crear el modelo Gemini  OPCION 1
modelo = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    temperature=0.2,
    model_kwargs={"tools": [calcular_raiz]}, # Langchain NO define tools como parámetro en su constructor, se recomienda esto como solución oficial
)

In [ ]:
# Invocar el modelo e imprimir resultado
# ---------------------------------------
respuesta = modelo.invoke("¿Cuánto es la raíz cuadrada de 16?")
print("RESPUESTA:", respuesta)

## 5. Tool compleja: conversión de moneda

In [ ]:
@tool
def usd_to_cop(cantidad_usd: float) -> float:
    '''Convierte USD a COP'''
    return cantidad_usd * 4200

In [ ]:
# Crear el modelo Gemini OPCION 2 (sin llamar la Tool usd_to_cop)
modelo = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    temperature=0.2,
)

In [ ]:
respuesta = modelo.invoke("Convierte 150 dólares a COP",tools=[usd_to_cop])

print("🔧 Tool call generada por el modelo:")
print(respuesta.tool_calls)

In [ ]:
for call in respuesta.tool_calls:
    if call["name"] == "usd_to_cop":
        final = usd_to_cop.invoke(call["args"])
        print("RESPUESTA:", final)